In [1]:
import os
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from scipy import linalg
import matplotlib.pyplot as plt
from scipy.stats import pearsonr, chi2
from sklearn.covariance import MinCovDet
from pandas.api.types import is_numeric_dtype
import utilities as utils

In [2]:
sns.set_theme(style="whitegrid")
plt.rcParams["figure.figsize"] = (12,8)
warnings.filterwarnings('ignore')

In [3]:
trainPath = 'train.csv'
target = 'SalePrice'
outdir = 'plots'

In [4]:
X = pd.read_csv(trainPath, index_col='Id')

In [ ]:
utils.plotAllFeatures(X, target, outdir=outdir)

In [20]:
dataSkewness = X.skew().sort_values(ascending=False)
skewed = dataSkewness[abs(dataSkewness) > 0.5]

realMult =['LotFrontage', 'LotArea', 'MasVnrArea',
            'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
                'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
                'LowQualFinSF', 'GrLivArea', 'GarageArea',
                'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch',
                '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal']

for a in realMult:
    if a not in skewed:
        print(a)

GarageArea


In [9]:
for name, data in X.iteritems():
    if is_numeric_dtype(data):
        print(data.skew())

1.4076567471495591
2.163569142324884
12.207687851233496
0.2169439277628693
0.6930674724842182
-0.613461172488183
-0.5035620027004709
2.669084210182863
1.685503071910789
4.255261108933303
0.9202684528039037
1.5242545490627664
1.3767566220336365
0.8130298163023265
9.011341288465387
1.3665603560164552
0.596066609663168
4.103402697955168
0.036561558402727165
0.675897448233722
0.21179009627507137
4.488396777072859
0.6763408364355531
0.6495651830548841
-0.6494146238714679
-0.3425489297486655
0.17998090674623907
1.5413757571931312
2.3643417403694404
3.08987190371177
10.304342032693112
4.122213743143115
14.828373640750588
24.476794188821916
0.21205298505146022
0.09626851386568028
1.8828757597682129


### Correlation analysis
- Correlate each numerical feature with the predictor.
- Save p-value, N and correlation coefficient for later.

In [ ]:
print(correlations.loc[
    (correlations['feature1'] != target) 
    & (correlations['feature2'] != target)
    & (correlations['feature1'] != correlations['feature2'])
    & (abs(correlations['R'] > 0.6))])

In [ ]:
correlations = utils.computeCorrelation(X)
utils.plotPairwiseCorrelation(
    correlations, out=f'{outdir}/pairwiseCorrelations.png')

In [ ]:
fig, (ax1, ax2) = utils.plotTargetCorrelation(
    correlations, target, out = f'{outdir}/{target}-correlations.png')

### Missing data assessment
- Compute NA frequency for each feature

In [ ]:
nullOrd = X.isnull().sum().sort_values(ascending=False)
nullOrd = nullOrd[nullOrd > 0]
nullOrd = pd.DataFrame(nullOrd, columns=['TotalNA'])
nullOrd['PropNA'] = nullOrd['TotalNA'] / len(X)
nullOrd

In [ ]:
fix, ax = plt.subplots()
sns.heatmap(X[nullOrd.index].isnull().T, yticklabels=1, cmap='viridis', cbar=False, xticklabels=False, ax=ax)
ax.set_xlabel('')
ax.tick_params(left=True)
fig.tight_layout()

## Missing values

### High proportion NA features
- Remove all features with > 15% missing values.
- No strong relationship with SalePrice anyway.

In [ ]:
# Processing
X['Alley'] = X['Alley'].fillna('No alley access')
X['BsmtQual'] = X['BsmtQual'].fillna('No Basement')
X['BsmtCond'] = X['BsmtCond'].fillna('No Basement')
X['BsmtExposure'] = X['BsmtExposure'].fillna('No Basement')
X['BsmtFinType1'] = X['BsmtFinType1'].fillna('No Basement')
X['BsmtFinType2'] = X['BsmtFinType2'].fillna('No Basement')
X['BsmtFullBath'] = X['BsmtFullBath'].fillna(0)
X['BsmtHalfBath'] = X['BsmtHalfBath'].fillna(0)
X['BsmtUnfSF'] = X['BsmtUnfSF'].fillna(0)
X['Fence'] = X['Fence'].fillna('No Fence' )
X['FireplaceQu'] = X['FireplaceQu'].fillna('No Fireplace')
X['Fireplaces'] = X['Fireplaces'].fillna(0)
X['MiscFeature'] = X['MiscFeature'].fillna('No misc feature')
X['MiscVal'] = X['MiscVal'].fillna(0)
X['Functional'] = X['Functional'].fillna('Typ')
X['GarageType'] = X['GarageType'].fillna('No garage')
X['GarageFinish'] = X['GarageFinish'].fillna('No garage')
X['GarageQual'] = X['GarageQual'].fillna('No garage')
X['GarageCond'] = X['GarageCond'].fillna('No garage')
X['GarageArea'] = X['GarageArea'].fillna(0)
X['GarageCars'] = X['GarageCars'].fillna(0)
X['PoolQC'] = X['PoolQC'].fillna('No pool')
X['PoolArea'] = X['PoolArea'].fillna(0)

### Save processed
- Remerge features and predictor.
- Ensure to filter predictor by remaining indexes in feature in case rows have been filtered.

In [ ]:
X.to_csv('trainFilt.csv')

## Inspecting categoricals

In [ ]:
catGroups = X.select_dtypes(exclude=['number'])
catDescribe = catGroups.describe().sort_values(by=['unique'], ascending=False, axis=1).T
catSmallest = catGroups.melt().groupby('variable').apply(lambda x: x.value_counts().min()).rename('smallestGroup')
catDescribe = pd.merge(catDescribe, catSmallest,
                       left_index=True, right_index=True)
catDescribe

In [ ]:
X['Neighborhood'] = X['Neighborhood'].replace(countsSum[countsSum < 0.25].index, 'other')